# Offline Fake 2d Dedector

Here is a brief overview of three applications used in this system.

The sample system will use the default setting included in the ``beamlime``.

## Data Feeder

``beamlime.offline.data_feeder.fake_2d_detector_img_generator`` is used as a data feeder.

Please see ``fake_2d_image.ipynb`` for more information.

## Data Reduction

``beamlime.offline.data_reduction.BeamLimeDataReductionApplication`` is used as a data reduction handling application.

<!-- TODO: Create a documentation about data reduction application and link it here. -->

## Visualization

``bealime.offline.visualization.RealtimePlot`` is used as a visualization application.

<!-- TODO: Create a documentation about visualization application and link it here. -->


In [ ]:
from beamlime.resources.generated.loaders import load_static_default_config
config = load_static_default_config()

## Application Configuration

Each application is configured under ``data-stream.applications`` in the configuration file.

``name``: Name of the application.
``data-handler``: Handler class that will be used for the application. The handler class should inherit ``BeamlimeApplicationInterface``.
``timeout``: How long the application wants to wait for receiving data or sending data.
``wait-interval``: How often does the application wants to check if it can resume the work.

``timeout`` and ``wait-interval`` can be used with ``async_timeout`` decorator for various Exceptions.

<!-- TODO: Remove this configuration description after writing a proper documentation. -->

In [ ]:
config['data-stream']['applications']

## Logging

If logger is not given, the system will initiate a logger with the name ``beamlime``.

In this example, we will use ``scipp`` logger to use log widget.

Note that if you want to set the log level to ``logging.DEBUG``(0),

you will need to set ``widget_handler.level`` to logging.DEBUG specifically since ``logger.setLevel`` will not be sufficient.

Any level above ``logging.INFO`` should be fine.

```
widget_handler.level = logging.DEBUG
logger.setLevel('DEBUG')
```

In [ ]:
import scipp as sc
%matplotlib widget

logger = sc.get_logger()
widget_handler = sc.logging.make_widget_handler()
logger.addHandler(widget_handler)
logger.setLevel('INFO')

Logs will be shown in the widget below.

In [ ]:
sc.display_logs()

## Instantiate the system

Now you create a system instance with the ``configuration`` (if not given, it'll use the default one) and the ``logger``.

It will then instantiate all the applications according to the configuration and get ready to start.

All applications are connected by ``Queue`` with each other, and the direction of the data-flow is downstream.

> ``data-feeder`` -> Queue -> ``data-reduction`` -> Queue -> ``visualization``

The ``data-feeder`` app will keep creating a frame of fake data and pushing it into the ``Queue``, 

and the ``data-reduction`` app will keep drawing the data from the ``Queue`` and reducing the data and pushing the result into the next ``Queue``,

and the ``visualization`` app will keep drawing the data from the ``Queue`` and update the plot.


In [ ]:
from beamlime.core.system import BeamlimeSystem

manager = BeamlimeSystem(config=config, logger=logger)

In the jupyter notebook, it is easy to mess up the async event loop, 

so instead of using ``run`` interface, we will create the task from the coroutine.

Once you start the coroutine, the logs will be collected in the widget above.

In [ ]:
tasks = manager.create_task()

In [ ]:
tasks

In [ ]:
list(manager.applications['visualisation']._instances.values())[0].figs['heatmap']

## Kill and repopulate the application instances.

Let's try run the coroutine again, and then you can see the plot is stacking the new ones on top of the old result.

In [ ]:
manager.create_task()

If you want a fresh start, you can kill the visualization instance and populate it again.

In [ ]:
manager.applications['data-reduction'].kill()
manager.applications['visualisation'].kill()
manager.applications['data-reduction'].populate()
manager.applications['visualisation'].populate()

In [ ]:
manager.create_task()

In [ ]:
list(manager.applications['visualisation']._instances.values())[0].figs['heatmap']